In [1]:
import numpy as np
from glob import glob
from bcg import load_bcg
from fs import load_ecg
import plotly.graph_objs as go
from plotly.offline import plot
%load_ext autoreload
%autoreload 2

In [2]:
data_dir = "/home/guo/physio/BCG_data/bcg_ecg_data/bcg_ecg_data(500HZ)/"

In [3]:
bcg_files = glob(data_dir+"*.txt", recursive=False)
ecg_files = glob(data_dir+"*.edf", recursive=False)

In [4]:
bcg_files.sort()
ecg_files.sort()

In [5]:
from bcg import load_bcg
import findpeaks3
import autopeaks
import pdb

def findpeaks_in_bcg_resp(signal):
    bcg_auto_peaks = autopeaks.AutoPeaks(thres=0.7,min_dist=180,buffer_size=1000)
    list(map(bcg_auto_peaks.findpeaks, signal))
    return bcg_auto_peaks.peak_indexes, bcg_auto_peaks.peak_values

def findpeaks_in_bcg(signal):
    bcg_auto_peaks = autopeaks.AutoPeaks(thres=0.45,min_dist=60,buffer_size=200)
    list(map(bcg_auto_peaks.findpeaks, signal))
    return bcg_auto_peaks.peak_indexes, bcg_auto_peaks.peak_values

def findpeaks_in_ecg(signal):
    ecg_auto_peaks = autopeaks.AutoPeaks(thres=0.7,min_dist=60,buffer_size=1000)
    list(map(ecg_auto_peaks.findpeaks, signal))
    return ecg_auto_peaks.peak_indexes, ecg_auto_peaks.peak_values

def findpeaks_for_bcg_and_ecg(bcg_file, ecg_file, offset=0, is_return=False):
    if offset >= 0:
        bcg_offset = offset
    else:
        bcg_offset = 0
    bcg = load_bcg(bcg_file, offset=bcg_offset,filter=False, notch=False)
    ecg = load_ecg(ecg_file, filter=True)
    if offset < 0:
        ecg = ecg[-offset:]
    #print(bcg[:100])
    start = 100000
    end = start + 400000
    #bcg = bcg[start:end]
    #ecg = ecg[start:end]
    bcg_100 = resample_poly(bcg, 1, 5)
    bcg = findpeaks3.transform_signal_by_energy(bcg_100)
    plot([go.Scatter(y=bcg_100,name="bcg_100"), go.Scatter(y=lowpass_filter(bcg_100),name="bcg_100_low")])
    bcg_low = lowpass_filter(bcg)

    ecg = resample_poly(ecg, 1, 5)
    bcg_peak_indices, bcg_peak_values = findpeaks_in_bcg(np.asarray(bcg))
    ecg_peak_indices, ecg_peak_values = findpeaks_in_ecg(ecg)
    resp_peak_indices, resp_peak_values = findpeaks_in_bcg_resp(np.asarray(bcg_low))
    #np.savetxt("glt.txt", np.diff(bcg_peak_indices[1:]), delimiter=',', fmt="%d")
    
    if is_return:
        return bcg_peak_indices, bcg_peak_intervals, ecg_peak_indices
    if True:
        plot(go.Figure(data=[go.Scatter(x=bcg_peak_indices[1:],
                                        y=np.diff(bcg_peak_indices),  
                                        mode="lines+markers",
                                        name="bcg_intervals",
                                        marker=dict(color="blue")),
                             go.Scatter(x=ecg_peak_indices[1:], 
                                        y=np.diff(ecg_peak_indices), 
                                        mode="lines+markers", 
                                        name="ecg_intervals",
                                        marker=dict(color="green")),],                       
                            #go.Scatter(x=ecg_peak_indices[1:], 
                            #            y=np.diff(ecg_peak_indices)+30, 
                            #            mode="lines", 
                            #            name="ecg_intervals_upper",
                            #            marker=dict(color="red")),
                            #go.Scatter(x=ecg_peak_indices[1:], 
                            #            y=np.diff(ecg_peak_indices)-30, 
                            #            mode="lines", 
                            #            name="ecg_intervals",
                            #            marker=dict(color="red")),],
                      layout=go.Layout(yaxis=dict(range=[250,1500]))),filename=bcg_file)
    if True:
        # resp
        plot([go.Scatter(y=bcg, name="energy"), 
              go.Scatter(y=bcg_low,name="bcg_low"),
              go.Scatter(x=resp_peak_indices,y=np.asarray(bcg_low)[resp_peak_indices],mode="markers")],filename="resp")
        plot([go.Scatter(x=resp_peak_indices[1:], y=6000/np.diff(resp_peak_indices),name="resp interval", mode="lines+markers")],filename="resp rate")
        
        plot([go.Scatter(y=bcg_100,name="bcg", marker=dict(color="blue")),
              #go.Scatter(y=np.diff(bcg), name="dff1", marker=dict(color="green")),
              #go.Scatter(y=np.abs(np.diff(bcg)), name="dff1_abs", marker=dict(color="red")),
              #go.Scatter(y=np.diff(np.abs(np.diff(bcg))), name="diff(dff1_abs)", marker=dict(color="yellow")),
              go.Scatter(x=bcg_peak_indices, y=bcg_100[bcg_peak_indices], mode="markers", name="peak"),
              #go.Scatter(x=extrema_indices,y=extrema_values,
              #           mode="markers",
              #           name="raw_extremas",
              #           marker=dict(color="orange")),              
              #go.Scatter(x=raw_peaks_indices,y=raw_peaks_indicators,
              #           mode="lines+markers",
              #           name="raw_peaks_indicator",
              #           marker=dict(color="red")),
              #go.Scatter(x=raw_peaks_indices,y=raw_peaks_scores,
              #           mode="lines+markers",
              #           name="raw_peaks_score",
              #           marker=dict(color="deepskyblue")),
              #go.Scatter(y=thres_abses,
              #           mode="lines",
              #           name="thres_abs",
              #           marker=dict(color="black")),
              go.Scatter(y=ecg[1:]/20,name="ecg", marker=dict(color="purple"))],filename=bcg_file)
        time.sleep(1)
        #pdb.set_trace()
        plot([go.Scatter(y=bcg,name="bcg", marker=dict(color="blue")),
              #go.Scatter(y=np.diff(bcg), name="dff1", marker=dict(color="green")),
              #go.Scatter(y=np.abs(np.diff(bcg)), name="dff1_abs", marker=dict(color="red")),
              #go.Scatter(y=np.diff(np.abs(np.diff(bcg))), name="diff(dff1_abs)", marker=dict(color="yellow")),
              go.Scatter(x=bcg_peak_indices, y=np.asarray(bcg)[bcg_peak_indices], mode="markers", name="peak"),
              #go.Scatter(x=extrema_indices,y=extrema_values,
              #           mode="markers",
              #           name="raw_extremas",
              #           marker=dict(color="orange")),              
              #go.Scatter(x=raw_peaks_indices,y=raw_peaks_indicators,
              #           mode="lines+markers",
              #           name="raw_peaks_indicator",
              #           marker=dict(color="red")),
              #go.Scatter(x=raw_peaks_indices,y=raw_peaks_scores,
              #           mode="lines+markers",
              #           name="raw_peaks_score",
              #           marker=dict(color="deepskyblue")),
              #go.Scatter(y=thres_abses,
              #           mode="lines",
              #           name="thres_abs",
              #           marker=dict(color="black")),
              go.Scatter(y=np.asarray(ecg)*400,name="ecg", marker=dict(color="purple"))])        

In [6]:
import time

def get_bcg_offset(bcg_file):
    offset = 0
    if "yjj" in bcg_file:
        offset = 39558-396
    elif "glt" in bcg_file:
        offset = 36621 + 39421 - 116
    elif "hwm" in bcg_file:
        offset = 1000 + 74953 - 169
    elif "lcc" in bcg_file:
        offset = 60000 + 8287 - 10
    elif "lifeng" in bcg_file:
        offset = 90000 - 25471 + 4396
    elif "zyh" in bcg_file:
        offset = 300000 - 14583
    elif "zcj" in bcg_file:
        offset = -10235 + 170
    elif "wkp" in bcg_file:
        offset = 5004
    return offset

for bcg_file,ecg_file in zip(bcg_files, ecg_files):
    if "glt" not in bcg_file:
        continue
    print(bcg_file)
    offset = get_bcg_offset(bcg_file)
    print("offset: {}".format(offset))
    findpeaks_for_bcg_and_ecg(bcg_file, ecg_file, offset=offset)

In [7]:
from fs import lowpass_filter

In [53]:
bcg_low = lowpass_filter(bcg)

NameError: name 'bcg' is not defined